# iTEBD algorithm for the Ising model

This code is based on a code presented by Frank Pollmann at the FOR 1807 Winter School 2018. The code has been partially modified and extended.

The iTEBD (imaginary Time Evolution Block Decimation) algorithm is a method to either evolve states in time or to minimize the energy.
In the following, we will use it to minimize the energy of the Ising Hamiltonian subject to a magnetic field in x- and z-direction:

$$H=\sum_{i} \sigma^z_i\sigma^z_{i+1}+h_x \sigma^x_i + h_z\sigma^z_i$$

In the algorithm we use the following order of dimensions for tensors: [physical, left, right], the order for diagonal matrices is [left,right].
Before we actually look into the code, here is a pictorial summary of the contractions in the algorithm.
The reading order is from left to right, top to bottom.

![Contractions in the iTEBD algorithm](img_notebook/iTEBD_contractions.png)

Import some packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.linalg import svd, expm

Definitions of the Pauli matrices

In [ ]:
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0.,-1.j], [1.j,0.]])
sz = np.array([[1., 0.], [0., -1.]])
s0 = np.eye(2)

Definition of the Hamilton operator and the imaginary time evolution operator that we need in the iTEBD step

In [ ]:
def init_ising(J, hx, hz, delta):
    """ Returns the Hamilton opertator and the time evolution operator"""
    d = 2
    H = -J * np.kron(sz, sz) - hx * (np.kron(sx, s0) + np.kron(s0, sx)) / 2. - hz * (
       np.kron(sz, s0) + np.kron(s0, sz)) / 2.
    H_bond=np.reshape(H,(d,d,d,d))
    U_bond=np.reshape(expm(-delta * H), (d, d, d, d))
    return U_bond, H_bond

Function that performs the sweep for a two-site system
This is the actual algorithm that we looked at in the lecture.

**TODO**:
- Have a look at the function sweep implemented below and try to understand why the tensors are contracted in this way

In [ ]:
def sweep(gamma, lambd, U_bond, chi):
    """ Perform one time evolution step and calculate the energy"""
    L = len(gamma)
    d = gamma[0].shape[0]
    Evec =[]
    for k in [0,1]:
        ia = k
        ib = np.mod(k + 1, L)

        # Construct theta matrix #
        # Contraction 1
        theta=np.tensordot(np.diag(lambd[ib]),gamma[ia],axes=(1,1))
        # Contraction 2
        theta=np.tensordot(theta,np.diag(lambd[ia]),axes=(2,0))
        # Contraction 3
        theta=np.tensordot(theta,gamma[ib],axes=(2,1))
        # Contraction 4
        theta=np.tensordot(theta,np.diag(lambd[ib]),axes=(3,0))
        
        # Apply the time evolution #
        theta=np.tensordot(theta,U_bond,axes=([1,2],[0,1]))

        # Reshape the tensor to do SVD
        theta=np.reshape(np.transpose(theta,(2,0,3,1)),(d*chi,d*chi))
        # Singular value decomposition #
        U, S, Vd = svd(theta, full_matrices=0, lapack_driver='gesvd')
        # (S is sorted descending)   
        V=Vd.T
        S = S[:chi]
        invsq = np.sqrt(sum(S**2))
        lambd[ia]=S/invsq
        
        lbinv=np.diag(lambd[ib]**(-1))
        U=np.reshape(U[:d*chi,:chi],(d,chi,chi))
        gamma[ia]=np.transpose(np.tensordot(lbinv,U,axes=(1,1)),(1,0,2))
        
        V=np.transpose(np.reshape(V[:d*chi,:chi],(d,chi,chi)),(0,2,1))
        gamma[ib]=np.tensordot(V,lbinv,axes=(2,0))
        
        Evec.append(-np.log(np.sum(theta**2))/delta/2.)
    return np.mean(Evec)

Convergence of energy for a given value of $\delta$
We want to see that the energy actually converges against the correct value.
The function "f" defined below calculates the exact solution for the energy for the given parameters.

**TODO**:
- Measure the energy in every step of the loop and store the values
- Plot the values of energy (and the correct value)

In [ ]:
%matplotlib notebook

chi=10
J=1. #Do not change this value, since it gives the relative energy scale
hx=0.5
hz=0.0 #Do not change this value since we cannot solve the model exactly otherwise
n_imaginary=300
delta=0.001

# Generate a random initial state
gamma=[np.random.rand(2,chi,chi)]*2
lambd=[np.random.rand(chi)]*2

N = int(n_imaginary / np.sqrt(delta))
U_bond, H_bond = init_ising(J, hx, hz, delta)
for i in range(N):
    #TODO: Measure here
    sweep(gamma, lambd, U_bond, chi)
def f(k, hx):
    return -np.sqrt(1 + hx**2 - 2 * hx * np.cos(k)) / np.pi
E0_exact = integrate.quad(f, 0, np.pi, args=(hx, ))[0]

#Do you want to plot all energy values or could there be a reason to leave out some of the values?
fig,ax0 = plt.subplots(1,1,figsize=(10, 5))
#TODO: Plot here
ax0.set_xlabel('iteration')
ax0.set_ylabel('E');

Convergence of the energy for varying values of $\delta$
In the next cell, we calculate the energy for different values of the timestep $\delta$.

**TODO**
- Evaluate the error of the energy after the last iteration of the iTEBD algorithm and store the values
- Plot the dependence of the error in energy on $\delta$

In [ ]:
chi=10
J=1. #Do not change this value, since it gives the relative energy scale
hx=0.5
hz=0.0
n_imaginary=100

%matplotlib notebook
"""run imaginary time evolution for the infinite ising chain with iTEBD"""
# Generate a random initial state
gamma=[np.random.rand(2,chi,chi)]*2
lambd=[np.random.rand(chi)]*2

#Use different values of delta to show the influence of different timesteps
deltavec=[0.1, 0.01, 0.001, 0.0001]
dEvec=[]
for delta in deltavec:
    N = int(n_imaginary / np.sqrt(delta))
    U_bond, H_bond = init_ising(J, hx, hz, delta)
    for i in range(N):
        #TODO: Store the energy (again)
        sweep(gamma, lambd, U_bond, chi)

    # Calculate exact groundstate energy
    if hz == 0 and np.abs(J) == 1:
        def f(k, hx):
            return -np.sqrt(1 + hx**2 - 2 * hx * np.cos(k)) / np.pi
        E0_exact = integrate.quad(f, 0, np.pi, args=(hx, ))[0]
        #TODO: Save the difference between exact energy and calculated energy
        #print("--> delta = {D:.6f}, E = {E:2.6f} (dE = {dE:2.2e})".format(D=delta, E=E,dE=E - E0_exact))
    #else:
        #print("--> delta = {D:.6f}, m = {m:2.6f}, E = {E:2.6f}".format(D=delta, m=np.abs(m), E=E))
        
if hz==0 and np.abs(J) == 1:
    fig, axes=plt.subplots()
    # TODO: Plot the energy. A loglog plot might be reasonable. 
    axes.set_xlabel(r'$\delta$')
    axes.set_ylabel(r'dE')